<h1>
AI Practical Homework5 - Q1
</h1>
<h2>
Particle Filtering Problem
</h2>
<b>
Benyamin Maleki 
</b> 

Reading data of radiostations from radio_stations.json.

In [2]:
import json

def read_radio_stations():
    with open("radio_stations.json", "r") as f:
        radio_stations = json.load(f)
    
    radio_station_keys = set()
    for name in radio_stations:
        radio_station_keys.add(name)

    return radio_stations, radio_station_keys

radio_stations, radio_station_keys = read_radio_stations()

Reading the input file.


In [4]:
import numpy as np

with open("input_sample.txt", "r") as f:
    initial_coord = np.array([float(x) for x in f.readline().strip().split(", ")])

    step_magnitude_distrib_param = float(f.readline().strip())
    cov_matrix_param = float(f.readline().strip())
    count_steps_agent = int(f.readline().strip())

    estimated_distance = {}
    for _ in range(len(radio_stations)):
        station_name = f.readline().strip()
        for __ in range(count_steps_agent):
            dist = float(f.readline().strip())
            if station_name not in estimated_distance:
                estimated_distance[station_name] = []
            estimated_distance[station_name].append(dist)

Modeling movement of the agent.


In [5]:
def get_movement_vector():
    magnitude = np.random.exponential(step_magnitude_distrib_param)
    angle_vertical = np.random.uniform(-np.pi/8, 0)
    angle_from_x_ax = np.random.uniform(-np.pi/8, np.pi/4)

    delta_x = magnitude * np.cos(angle_vertical) * np.cos(angle_from_x_ax)
    delta_y = magnitude * np.cos(angle_vertical) * np.sin(angle_from_x_ax)
    delta_z = magnitude * np.sin(angle_vertical)
    return np.array([delta_x, delta_y, delta_z])

def get_cov_matrix():
    return np.identity(3) * cov_matrix_param


Now, we will perform particle filtering on the data

In [64]:
def assign_weight_to_particles(particles, step, estimated_distances):
    weights = np.zeros(particles.shape[0])
    for station_name in radio_station_keys:
        # No data of current step movement!
        if estimated_distances[station_name][step] == "None":
            continue
        # Assign weight (probability) to each particle
        for i in range(particles.shape[0]):
            estimated_distance = estimated_distances[station_name][step]
            particle_distance = np.linalg.norm(particles[i] - radio_stations[station_name]['coordinates'])
            weights[i] += np.exp(-particle_distance / estimated_distance)
    return weights

def particle_filtering():
    particle_count = 100
    particles = np.random.multivariate_normal(initial_coord, get_cov_matrix(), particle_count)
    for i in range(count_steps_agent):
        particles += get_movement_vector()
        # resample based on evidence
        weights = assign_weight_to_particles(particles, i, estimated_distance)
        weights /= weights.sum()
        chosen_particle_indices = np.random.choice(np.arange(particle_count), size=particle_count, p=weights)
        particles = particles[chosen_particle_indices]
    return particles

particles = particle_filtering()
mean = particles.mean(axis=0)
x = int(np.ceil(mean[0]/100)*100)
y = int(np.ceil(mean[1]/100)*100)
z = int(np.ceil(mean[2]/100)*100)
estimated_final_coord = np.array([x, y, z])
print("Mean: {}".format(estimated_final_coord))

Mean: [1400  300  200]
